# Adaptive Cards Toolkit: 03 - Data Visualization

This notebook demonstrates how to create cards with data visualizations like charts and tables. Please run 01_toolkit_common.ipynb first.

## Setup

First, we need to import the common utilities defined in the toolkit_common notebook as well as all visualization dependencies.

In [ ]:
# Run common utilities notebook (optional, can be skipped if already run in session)
%run 01_toolkit_common.ipynb

# Ensure we have all the necessary imports
import json
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add the parent directory to the path so we can import the toolkit
if not os.path.abspath('..') in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

# Import required modules directly
from src.adaptive_cards_toolkit.core.element_factory import ElementFactory
from src.adaptive_cards_toolkit.core.layout_helper import LayoutHelper
from src.adaptive_cards_toolkit.core.data_connector import DataConnector

# Define fig_to_base64 function for converting plots to base64 for embedding in cards
from io import BytesIO
import base64

def fig_to_base64(fig):
    """Convert a matplotlib figure to a base64 encoded string.
    
    Args:
        fig: Matplotlib figure object
        
    Returns:
        String with data URI containing base64 encoded image
    """
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=100)
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    return f"data:image/png;base64,{img_str}"

# Utility to display card
def display_card(card, title="Adaptive Card"):
    """Display an adaptive card object
    
    Args:
        card: Card object with to_json method
        title: Title to display with the card
    """
    print(f"\n== {title} ==\n")
    try:
        # Get JSON from the card
        if hasattr(card, 'to_json'):
            card_json = card.to_json()
        elif hasattr(card, 'card_data'):
            card_json = json.dumps(card.card_data, indent=2)
        else:
            card_json = json.dumps(card, indent=2)
            
        # Print the JSON
        print(card_json)
    except Exception as e:
        print(f"Error displaying card: {e}")

## Create Sample Data

First, let's create some sample sales data for our visualization.

In [ ]:
# Create sample data - sales performance by region
regional_data = {
    "regions": ["North", "South", "East", "West", "Central"],
    "sales": [342000, 287000, 411000, 394000, 218000],
    "target": [350000, 275000, 400000, 425000, 225000],
    "reps": [12, 9, 15, 14, 8]
}

# Convert to pandas DataFrame
df = pd.DataFrame(regional_data)

# Calculate whether targets were met and percentage of target
df["target_met"] = df["sales"] >= df["target"]
df["percent_of_target"] = (df["sales"] / df["target"] * 100).round(1)
df["sales_per_rep"] = (df["sales"] / df["reps"]).round(0)

# Display the dataframe
df

## Create Primary Visualization

Let's create a bar chart showing sales performance by region compared to targets.

In [ ]:
# Create primary visualization showing sales vs targets
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

# Create bar plot comparing sales by region
palette = ["green" if met else "red" for met in df["target_met"]]
bar_plot = sns.barplot(
    x="regions", 
    y="sales", 
    data=df,
    palette=palette,
    hue="regions",
    legend=False
)

# Add target markers
for i, target in enumerate(df["target"]):
    plt.plot([i-0.4, i+0.4], [target, target], color="black", linestyle="--")

plt.title("Sales vs. Target by Region")
plt.ylabel("Sales ($)")
plt.tight_layout()

# Display the plot
plt.show()

# Convert to base64 for embedding in card
chart_img = fig_to_base64(plt.gcf())
plt.close()

## Create Secondary Visualization

Now let's create a second visualization showing sales per representative.

In [ ]:
# Create a secondary visualization for sales per rep
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

# Create horizontal bar chart for sales per rep
bar_plot = sns.barplot(
    x="sales_per_rep",
    y="regions",
    data=df,
    palette="viridis",
    orient="h"
)

# Add average line
avg_sales_per_rep = df["sales_per_rep"].mean()
plt.axvline(x=avg_sales_per_rep, color="red", linestyle="--")
plt.text(avg_sales_per_rep+1000, 0.5, f"Avg: ${avg_sales_per_rep:,.0f}", color="red")

plt.title("Sales per Representative by Region")
plt.xlabel("Sales per Rep ($)")
plt.tight_layout()

# Display the plot
plt.show()

# Convert to base64 for embedding in card
secondary_chart_img = fig_to_base64(plt.gcf())
plt.close()

## Create the Data Visualization Card

Let's create a card with both visualizations and tabular data following the project's JSON-based card creation pattern.

In [ ]:
# Create card with JSON construction pattern from style guide
card_data = {
    "type": "AdaptiveCard",
    "version": "1.5",
    "body": [
        # Title and description
        {"type": "TextBlock", "text": "Regional Sales Performance", "size": "large", "weight": "bolder"},
        {"type": "TextBlock", "text": "Q2 2024 sales performance by region compared to targets.", "wrap": True},
        
        # Primary chart
        {"type": "TextBlock", "text": "Sales Performance by Region", "size": "medium", "weight": "bolder"},
        {"type": "Image", "url": chart_img, "altText": "Bar chart showing sales vs target by region"},
        
        # Secondary chart
        {"type": "TextBlock", "text": "Sales per Representative", "size": "medium", "weight": "bolder"},
        {"type": "Image", "url": secondary_chart_img, "altText": "Bar chart showing sales per representative by region"}
    ]
}

In [ ]:
# Add table header
card_data["body"].append({"type": "TextBlock", "text": "Detailed Performance Data", "size": "medium", "weight": "bolder"})
card_data["body"].append({
    "type": "ColumnSet", 
    "columns": [
        {"type": "Column", "width": "stretch", "items": [
            {"type": "TextBlock", "text": "Region", "weight": "bolder"}
        ]},
        {"type": "Column", "width": "stretch", "items": [
            {"type": "TextBlock", "text": "Sales ($)", "weight": "bolder"}
        ]},
        {"type": "Column", "width": "stretch", "items": [
            {"type": "TextBlock", "text": "Target ($)", "weight": "bolder"}
        ]},
        {"type": "Column", "width": "stretch", "items": [
            {"type": "TextBlock", "text": "% of Target", "weight": "bolder"}
        ]},
        {"type": "Column", "width": "stretch", "items": [
            {"type": "TextBlock", "text": "Sales per Rep", "weight": "bolder"}
        ]}
    ]
})

In [ ]:
# Add table rows
for i, row in df.iterrows():
    # Use different background for alternating rows
    bg_color = "default" if i % 2 == 0 else "accent"
    column_set = {
        "type": "ColumnSet", 
        "style": bg_color,
        "columns": [
            {"type": "Column", "width": "stretch", "items": [
                {"type": "TextBlock", "text": row["regions"]}
            ]},
            {"type": "Column", "width": "stretch", "items": [
                {"type": "TextBlock", "text": f"${row['sales']:,.0f}"}
            ]},
            {"type": "Column", "width": "stretch", "items": [
                {"type": "TextBlock", "text": f"${row['target']:,.0f}"}
            ]},
            {"type": "Column", "width": "stretch", "items": [
                {"type": "TextBlock", 
                 "text": f"{row['percent_of_target']}%", 
                 "color": "good" if row["target_met"] else "warning"}
            ]},
            {"type": "Column", "width": "stretch", "items": [
                {"type": "TextBlock", "text": f"${row['sales_per_rep']:,.0f}"}
            ]}
        ]
    }
    card_data["body"].append(column_set)

In [ ]:
# Add insights section
# Generate insights
best_region = df.loc[df["sales"].idxmax(), "regions"]
worst_region = df.loc[df["sales"].idxmin(), "regions"]
best_performer = df.loc[df["sales_per_rep"].idxmax(), "regions"]
regions_over_target = df[df["target_met"] == True].shape[0]
total_regions = df.shape[0]

card_data["body"].append({"type": "TextBlock", "text": "Key Takeaways", "size": "medium", "weight": "bolder"})

insights = [
    f"• **{best_region}** had the highest total sales at ${df['sales'].max():,}.",
    f"• **{worst_region}** had the lowest total sales at ${df['sales'].min():,}.",
    f"• **{best_performer}** had the highest sales per representative at ${df['sales_per_rep'].max():,.0f}.",
    f"• {regions_over_target} out of {total_regions} regions met or exceeded their sales targets."
]

for insight in insights:
    card_data["body"].append({"type": "TextBlock", "text": insight, "wrap": True})

In [ ]:
# Add action buttons
card_data["actions"] = [
    {
        "type": "Action.Submit",
        "title": "Download Full Report",
        "data": {"action": "download_report"}
    },
    {
        "type": "Action.Submit",
        "title": "Schedule Review Meeting",
        "data": {"action": "schedule_meeting"}
    }
]

# Create a card object with to_json method as per style guide
class CardObject:
    def __init__(self, card_data):
        self.card_data = card_data
        
    def to_json(self):
        return json.dumps(self.card_data)

# Create the card object
data_card = CardObject(card_data)

# Display the card
display_card(data_card, "Data Visualization Card")